In [1]:
import sys
print(sys.executable)

/usr/local/bin/python3


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.signal import butter, lfilter
from scipy.signal import freqz

In [4]:
subjectData = {}

In [5]:
ns = 10

# data_path = os.path.join("A{:02d}T.npz".format(1))
for i in range(1,ns):
    data_path = os.path.join("A{:02d}T.npz".format(i))
    subject = 'subject{:02d}'.format(i)

    subjectData[subject] = np.load(data_path)

In [6]:
print(type(subjectData['subject01']))
print(subjectData['subject01'].files)


<class 'numpy.lib.npyio.NpzFile'>
['s', 'etyp', 'epos', 'edur', 'artifacts']


s: contains time-series recorded EEG signals of shape MxN array. N is the number of electrodes (22 EEG and 3 EOG), M can vary

etpye: event type to indicate event occurence

epos: event position denoting event start at s


edur: event duration artifacts: size of 288x1, 6x48 = 288 where 6 is the number of runs with 48 trials. 48 trials have 12 trials of 4 class each

In [7]:
print('Sample\t Electrodes')
for i in range(1,ns):
    sub = 'subject{:02d}'.format(i)
    print('subject{:02d}'.format(i), subjectData[sub]['s'].shape)

Sample	 Electrodes
subject01 (672528, 25)
subject02 (677169, 25)
subject03 (660530, 25)
subject04 (600915, 25)
subject05 (686120, 25)
subject06 (678980, 25)
subject07 (681071, 25)
subject08 (675270, 25)
subject09 (673328, 25)


In [6]:
# Bandpass filter
def butter_bandpass_filter(signal, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut/nyq
    high = highcut/nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b,a,signal, axis=-1)
    return y

In [7]:
def butterFilter(cleanData, data, sub):
    lowcut = 4
    highcut = 40
    fs = 250
    print('Processing', sub)
    cleanData[sub] = butter_bandpass_filter(data, lowcut, highcut, fs)

In [14]:
# Transpose the raw EEG data to apply filtering on the time-series axis
rawData = {}
for i in range(1,ns):
    sub = 'subject{:02d}'.format(i)
    rawData[sub] = subjectData[sub]['s'].T

    print('subject{:02d}'.format(i), rawData[sub].shape)

subject01 (25, 672528)
subject02 (25, 677169)
subject03 (25, 660530)
subject04 (25, 600915)
subject05 (25, 686120)
subject06 (25, 678980)
subject07 (25, 681071)
subject08 (25, 675270)
subject09 (25, 673328)


In [16]:
cleanEEG = {}
for i in range(1,ns):
    sub = 'subject{:02d}'.format(i)
    butterFilter(cleanEEG, rawData[sub], sub)

    print(cleanEEG[sub].shape)

Processing subject01
(25, 672528)
Processing subject02
(25, 677169)
Processing subject03
(25, 660530)
Processing subject04
(25, 600915)
Processing subject05
(25, 686120)
Processing subject06
(25, 678980)
Processing subject07
(25, 681071)
Processing subject08
(25, 675270)
Processing subject09
(25, 673328)


There are 4 classes with event code as 769, 770, 771, 772 for left, right, foot and tongue.

In [8]:
left_class_code = 769
right_class_code = 770
foot_class_code = 771
tongue_class_code = 772

for i in range(1,2):
    sub = 'subject{:02d}'.format(i)
    left = subjectData[sub]['epos'][subjectData[sub]['etyp'] == left_class_code]
    right = subjectData[sub]['epos'][subjectData[sub]['etyp'] == right_class_code]
    foot = subjectData[sub]['epos'][subjectData[sub]['etyp'] == foot_class_code]
    tongue = subjectData[sub]['epos'][subjectData[sub]['etyp'] == tongue_class_code]



In [9]:
right

array([ 96290, 102361, 108259, 119941, 122061, 132307, 146352, 152424,
       156623, 172782, 176750, 182703, 195077, 205094, 218896, 233134,
       237118, 243187, 259656, 263648, 267536, 273585, 275704, 277610,
       286039, 303927, 311644, 317739, 328057, 335914, 338018, 344023,
       352384, 358511, 360483, 378487, 382874, 392866, 406535, 412566,
       424892, 426804, 432749, 434853, 436857, 461206, 471280, 475322,
       485582, 497597, 501398, 503370, 507281, 513475, 515511, 517577,
       519647, 531688, 535667, 560122, 580465, 594432, 602149, 604116,
       606236, 610310, 618562, 638706, 640798, 658935, 660925, 668992],
      dtype=int32)

These are the sample points where the event has started

Steps to start with

1. Remove EOG channels from all subjects

2. Crop the data for each subject such that you get the data during the motor imagery peroid (refer Experimental_Paradigm)


In [10]:
print(subjectData['subject02'].files)

print(len(subjectData['subject02']['s']))
print(len(subjectData['subject02']['etyp']))
print(len(subjectData['subject02']['epos']))
print(len(subjectData['subject02']['edur']))
print((subjectData['subject02']['etyp']))


['s', 'etyp', 'epos', 'edur', 'artifacts']
677169
606
606
606
[[32766]
 [  276]
 [32766]
 [  277]
 [32766]
 [ 1072]
 [32766]
 [  768]
 [  769]
 [  768]
 [  770]
 [  768]
 [  770]
 [  768]
 [  769]
 [  768]
 [  770]
 [  768]
 [  769]
 [  768]
 [  770]
 [  768]
 [  771]
 [  768]
 [  770]
 [  768]
 [  772]
 [  768]
 [  769]
 [  768]
 [  771]
 [  768]
 [  770]
 [  768]
 [  769]
 [  768]
 [  772]
 [  768]
 [  772]
 [  768]
 [  772]
 [  768]
 [  772]
 [  768]
 [  772]
 [  768]
 [  769]
 [  768]
 [  771]
 [  768]
 [  770]
 [  768]
 [  769]
 [  768]
 [  769]
 [  768]
 [  771]
 [  768]
 [  772]
 [  768]
 [  769]
 [  768]
 [  771]
 [  768]
 [  771]
 [  768]
 [  771]
 [  768]
 [  769]
 [  768]
 [  770]
 [  768]
 [  769]
 [  768]
 [  770]
 [  768]
 [  770]
 [  768]
 [  769]
 [  768]
 [  770]
 [  768]
 [  771]
 [  768]
 [  770]
 [  768]
 [  771]
 [  768]
 [  771]
 [  768]
 [  772]
 [  768]
 [  771]
 [  768]
 [  771]
 [  768]
 [  772]
 [  768]
 [  772]
 [  768]
 [  772]
 [  768]
 [  772]
 [32766]
 [